<a href="https://colab.research.google.com/github/FestusMikhael/DeepLearning_EksplorasiResNet-34/blob/main/Tahap_1_Plain_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchinfo

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

class PlainBlock(nn.Module):
    """
    Plain Block without residual connection.
    This is equivalent to a ResNet BasicBlock but without the skip connection.
    """
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(PlainBlock, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Downsample layer for dimension matching (if needed)
        self.downsample = downsample

    def forward(self, x):
        # Store input for potential downsampling
        identity = x

        # First conv + bn + relu
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        # Second conv + bn
        out = self.conv2(out)
        out = self.bn2(out)

        # Apply downsample to identity if needed (for dimension matching)
        if self.downsample is not None:
            identity = self.downsample(identity)

        # NO RESIDUAL CONNECTION HERE (this is the key difference from ResNet)
        # In ResNet, we would do: out += identity
        # But in Plain network, we just apply ReLU directly

        out = F.relu(out)

        return out

class Plain34(nn.Module):
    """
    Plain-34 Network: ResNet-34 architecture without residual connections.

    Architecture:
    - Initial conv layer (7x7, stride=2)
    - MaxPool (3x3, stride=2)
    - 4 stages of Plain blocks:
      - Stage 1: 3 blocks, 64 channels
      - Stage 2: 4 blocks, 128 channels, stride=2 for first block
      - Stage 3: 6 blocks, 256 channels, stride=2 for first block
      - Stage 4: 3 blocks, 512 channels, stride=2 for first block
    - Global Average Pool
    - Fully Connected layer
    """

    def __init__(self, num_classes=5):
        super(Plain34, self).__init__()

        # Initial convolutional layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Plain block stages
        self.stage1 = self._make_stage(64, 64, 3, stride=1)    # 3 blocks, 64 channels
        self.stage2 = self._make_stage(64, 128, 4, stride=2)   # 4 blocks, 128 channels
        self.stage3 = self._make_stage(128, 256, 6, stride=2)  # 6 blocks, 256 channels
        self.stage4 = self._make_stage(256, 512, 3, stride=2)  # 3 blocks, 512 channels

        # Final layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

        # Initialize weights
        self._initialize_weights()

    def _make_stage(self, in_channels, out_channels, num_blocks, stride):
        """
        Create a stage consisting of multiple PlainBlocks.

        Args:
            in_channels: Input channels for the first block
            out_channels: Output channels for all blocks in this stage
            num_blocks: Number of blocks in this stage
            stride: Stride for the first block (usually 1 or 2)
        """
        downsample = None

        # If we need to change dimensions or stride, create downsample layer
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                         stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

        layers = []

        # First block (may have stride=2 and different input/output channels)
        layers.append(PlainBlock(in_channels, out_channels, stride, downsample))

        # Remaining blocks (stride=1, same input/output channels)
        for _ in range(1, num_blocks):
            layers.append(PlainBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

    def _initialize_weights(self):
        """Initialize model weights using He initialization."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Initial conv + bn + relu + maxpool
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)

        # Plain block stages
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)

        # Final classification layers
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def create_plain34(num_classes=5):
    """
    Factory function to create Plain-34 model.

    Args:
        num_classes: Number of output classes (default: 5 for Indonesian food dataset)

    Returns:
        Plain34 model instance
    """
    return Plain34(num_classes=num_classes)

def test_model():
    """
    Test function to verify the model works correctly.
    This function creates a model and prints its architecture summary.
    """
    print("Creating Plain-34 model...")
    model = create_plain34(num_classes=5)

    # Print model summary
    print("\n" + "="*50)
    print("PLAIN-34 MODEL ARCHITECTURE SUMMARY")
    print("="*50)

    # Test with typical input size for image classification (224x224)
    try:
        summary(model, input_size=(1, 3, 224, 224), verbose=1)
    except Exception as e:
        print(f"Error in torchinfo summary: {e}")
        print("Trying manual forward pass...")

        # Manual test
        model.eval()
        with torch.no_grad():
            test_input = torch.randn(1, 3, 224, 224)
            output = model(test_input)
            print(f"Input shape: {test_input.shape}")
            print(f"Output shape: {output.shape}")
            print(f"Expected output shape: (1, 5)")
            print(f"Model works correctly: {output.shape == (1, 5)}")

    # Count total parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"\nTotal parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

    return model

if __name__ == "__main__":
    # Test the model when running this file directly
    model = test_model()

    print("\n" + "="*50)
    print("MODEL READY FOR TRAINING!")
    print("="*50)
    print("Next steps:")
    print("1. Load your Indonesian food dataset")
    print("2. Set up data loaders")
    print("3. Define loss function and optimizer")
    print("4. Train the model")
    print("5. Compare with ResNet-34 (with residual connections)")

Creating Plain-34 model...

PLAIN-34 MODEL ARCHITECTURE SUMMARY
Layer (type:depth-idx)                   Output Shape              Param #
Plain34                                  [1, 5]                    --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─MaxPool2d: 1-3                         [1, 64, 56, 56]           --
├─Sequential: 1-4                        [1, 64, 56, 56]           --
│    └─PlainBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─Conv2d: 3-3                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-4             [1, 64, 56, 56]           128
│    └─PlainBlock: 2-2                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-5                  [1, 64, 56, 56]           36,864
│  

In [4]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
import pandas as pd
from google.colab import drive

# 1. MOUNT GOOGLE DRIVE
drive.mount('/content/drive')
print("Google Drive berhasil di-mount.")

# 2. KONFIGURASI HYPERPARAMETER
HYPERPARAMS = {
    'learning_rate': 0.01,
    'batch_size': 16,
    'num_epochs': 10,
    'momentum': 0.9,
    'weight_decay': 1e-4,
    'num_classes': 5,
    'input_size': 224,
    'dataset_path': '/content/drive/MyDrive/IF25-4041-dataset', # PATH ANDA
}

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {DEVICE}")

RESULTS_DIR = '/content/drive/MyDrive/Model_Results'
os.makedirs(RESULTS_DIR, exist_ok=True)

# 3. TRANSFORMASI DATA
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(HYPERPARAMS['input_size']),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(HYPERPARAMS['input_size']),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 4. CUSTOM DATASET (MEMBACA DARI CSV)
class IndonesianFoodDataset(Dataset):
    """Membaca file gambar dan label dari CSV."""
    def __init__(self, data_root, csv_file, transform=None):
        self.data_root = data_root
        self.transform = transform

        # Baca CSV
        self.data_frame = pd.read_csv(csv_file)

        # ASUMSI KOLOM CSV: 'filename' dan 'label'. Sesuaikan jika beda!
        self.filenames = self.data_frame['filename'].values
        self.labels = self.data_frame['label'].values

        # Buat pemetaan label string ke integer (0, 1, 2, 3, 4)
        unique_labels = sorted(self.data_frame['label'].unique())
        self.label_to_idx = {label: i for i, label in enumerate(unique_labels)}

        # Konversi label string di DataFrame menjadi integer index
        self.labels = [self.label_to_idx[label] for label in self.labels]

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.filenames[idx]
        label = self.labels[idx]

        img_path = os.path.join(self.data_root, img_name)

        try:
            image = Image.open(img_path).convert('RGB')
        except FileNotFoundError:
            # Jika gambar tidak ada (misalnya ada di CSV tapi file tidak ada)
            print(f"File tidak ditemukan: {img_path}. Melewatkan.")
            return None, None

        if self.transform:
            image = self.transform(image)

        return image, label

# 5. MUAT DATASET DAN SPLIT (Train Asli -> Train & Val)
TRAIN_DIR = os.path.join(HYPERPARAMS['dataset_path'], 'train')
TRAIN_CSV = os.path.join(HYPERPARAMS['dataset_path'], 'train.csv')

try:
    # Muat SELURUH data training
    full_train_dataset = IndonesianFoodDataset(
        data_root=TRAIN_DIR,
        csv_file=TRAIN_CSV,
        transform=data_transforms['train']
    )

    if len(full_train_dataset) == 0:
        raise ValueError("Dataset 'train' kosong. Cek 'train.csv' dan folder 'train'.")

    # Split 85% Train, 15% Val
    train_size = int(0.85 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size

    train_dataset, val_dataset = random_split(
        full_train_dataset, [train_size, val_size]
    )

    train_loader = DataLoader(train_dataset, batch_size=HYPERPARAMS['batch_size'], shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=HYPERPARAMS['batch_size'], shuffle=False, num_workers=2)

    print("\n--- Data Loading Berhasil ---")
    print(f"Total Data: {len(full_train_dataset)}")
    print(f"Data Train (85%): {len(train_dataset)}")
    print(f"Data Val (15%): {len(val_dataset)}")

except ValueError as e:
    print(f"\n[ERROR KRITIS] {e}")
    print(f"Cek path: {TRAIN_CSV} dan {TRAIN_DIR}")
except Exception as e:
    print(f"\n[ERROR UMUM] Gagal memuat data: {e}")

print("Cell 2: Setup dan Data Loading Selesai.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive berhasil di-mount.
Menggunakan device: cuda:0

--- Data Loading Berhasil ---
Total Data: 1108
Data Train (85%): 941
Data Val (15%): 167
Cell 2: Setup dan Data Loading Selesai.


In [5]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import pandas as pd
import time

# --- 1. FUNGSI PELATIHAN INTU ---
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    start_time = time.time()
    history = pd.DataFrame(columns=['epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc'])

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print('-' * 10)

        # FASE TRAINING
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        scheduler.step()

        epoch_loss_train = running_loss / len(train_dataset)
        epoch_acc_train = running_corrects.double() / len(train_dataset)

        # FASE VALIDASI
        model.eval()
        running_loss_val = 0.0
        running_corrects_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss_val += loss.item() * inputs.size(0)
                running_corrects_val += torch.sum(preds == labels.data)

        epoch_loss_val = running_loss_val / len(val_dataset)
        epoch_acc_val = running_corrects_val.double() / len(val_dataset)

        print(f"Train Loss: {epoch_loss_train:.4f} | Train Acc: {epoch_acc_train:.4f}")
        print(f"Val Loss: {epoch_loss_val:.4f} | Val Acc: {epoch_acc_val:.4f}")

        history.loc[epoch] = [epoch + 1, epoch_loss_train, epoch_acc_train.item(), epoch_loss_val, epoch_acc_val.item()]

    time_elapsed = time.time() - start_time
    print(f'\nPelatihan selesai dalam {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model, history

# --- 2. INISIALISASI DAN EKSEKUSI PELATIHAN ---
print("\n" + "="*50)
print("MEMULAI PELATIHAN PLAIN-34 (BASELINE)")
print("="*50)

# Inisialisasi Model
model = create_plain34(num_classes=HYPERPARAMS['num_classes'])
model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=HYPERPARAMS['learning_rate'],
    momentum=HYPERPARAMS['momentum'],
    weight_decay=HYPERPARAMS['weight_decay']
)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Latih model
model_plain, history_plain = train_model(
    model,
    criterion,
    optimizer,
    scheduler,
    num_epochs=HYPERPARAMS['num_epochs']
)

# Penyimpanan Hasil
history_path = os.path.join(RESULTS_DIR, 'plain34_baseline_history.csv')
history_plain.to_csv(history_path, index=False)
torch.save(model_plain.state_dict(), os.path.join(RESULTS_DIR, 'plain34.pth'))

print("\n" + "="*50)
print("PELATIHAN BASELINE SELESAI!")
print(f"History dan bobot model disimpan di: {RESULTS_DIR}")
print("="*50)


MEMULAI PELATIHAN PLAIN-34 (BASELINE)

Epoch 1/10
----------


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss: 1.6745 | Train Acc: 0.2115
Val Loss: 1.8465 | Val Acc: 0.2934

Epoch 2/10
----------


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss: 1.5764 | Train Acc: 0.2795
Val Loss: 1.4248 | Val Acc: 0.3533

Epoch 3/10
----------


...

Train Loss: 1.3335 | Train Acc: 0.4251
Val Loss: 1.3003 | Val Acc: 0.4371

Epoch 9/10
----------


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss: 1.3039 | Train Acc: 0.4251
Val Loss: 1.2091 | Val Acc: 0.4671

Epoch 10/10
----------


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss: 1.2189 | Train Acc: 0.4729
Val Loss: 1.1985 | Val Acc: 0.4850

Pelatihan selesai dalam 8m 23s

PELATIHAN BASELINE SELESAI!
History dan bobot model disimpan di: /content/drive/MyDrive/Model_Results
